In [1]:
import contextlib


def make_val_and_grad_fn(value_fn):
  @functools.wraps(value_fn)
  def val_and_grad(x):
    return tfp.math.value_and_gradient(value_fn, x)
  return val_and_grad



@contextlib.contextmanager
def timed_execution():
  t0 = time.time()
  yield
  dt = time.time() - t0
  print('Evaluation took: %f seconds' % dt)


def np_value(tensor):
  """Get numpy value out of possibly nested tuple of tensors."""
  if isinstance(tensor, tuple):
    return type(tensor)(*(np_value(t) for t in tensor))
  else:
    return tensor.numpy()

def run(optimizer):
  """Run an optimizer and measure it's evaluation time."""
  optimizer()  # Warmup.
  with timed_execution():
    result = optimizer()
  return np_value(result)

In [2]:
import numpy as np
from math import gamma
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import time
import pandas as pd

import os
os.chdir("C:\\Users\\user\\Downloads\\Thesis_paper_1\\calibration")

np.set_printoptions(suppress=True) # supress scientific notation

import matplotlib.ticker as mtick
import locale
locale.setlocale(locale.LC_NUMERIC, "de_DE")


plt.rcParams['axes.formatter.use_locale'] = True



pd.set_option('display.max_columns', 50) 

In [3]:

import tensorflow as tf
import tensorflow_probability as tfp

import functools

from scipy.optimize import differential_evolution
from scipy.optimize import basinhopping
from scipy.optimize import shgo

In [4]:
## parameters



def get_data(year):
    w='data_' + str(year)
    alfa = tf.convert_to_tensor(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,5], dtype='float64').reshape(1,3) )
    phi = tf.convert_to_tensor(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,4], dtype='float64').reshape(1,3) )
    return alfa, phi



def pars(year):
    global alfa, rho, beta, theta, eta, psi, kappa, lamb, gamma1, G, phi, s
    rho = tf.constant(0.19, tf.float64)
    beta = tf.constant(0.231, tf.float64)
    theta = tf.constant(2.0, tf.float64)
    eta = tf.constant(0.103, tf.float64)
    psi = eta**eta*(1.0-eta)**(1.0-eta)
    kappa = tf.divide(1.0, (1.0 - eta))
    lamb = 1 - tf.divide(kappa, theta)
    gamma1 = tf.multiply(eta, kappa)    
    G = tf.constant(gamma(1 - tf.divide(kappa, tf.multiply(theta, (1-rho)) ) ), tf.float64  )      
    alfa, phi = get_data(year)
    s = tf.pow( (1 + tf.divide( (1-eta), ( tf.multiply(beta, phi) ) ) ), -1 )


pars(15)

In [5]:

df = pd.DataFrame()

l = [92, 95, 2, 15]

for i in l:
    w='data_' + str(i)
    k ='alfa_' + str(i)
    df[k] = np.array(pd.read_excel(f'{w}.xlsx').iloc[:,5], dtype='float64').reshape(3)



df

In [6]:
df2 = pd.DataFrame()


for i in l:
    w='data_' + str(i)
    j ='phi_' + str(i)
    df2[j] = np.array(pd.read_excel(f'{w}.xlsx').iloc[:,4], dtype='float64').reshape(3)

    
df2

In [7]:
# taus
st=3
def taus(st):        
    x1 = tf.random.uniform(maxval=10, minval=-10, shape=[1, st], dtype='float64')
    return x1

In [8]:
def N_if(x1):
    S = tf.pow(s, phi*(theta*lamb+ kappa) )
    S2 =tf.pow((1 - s), ( (theta*lamb)/(beta*kappa) ) )
    H = tf.pow((1 + x1[0]), (theta*lamb + kappa))    
    C = tf.multiply(tf.multiply(tf.pow(psi, theta*lamb), eta**gamma1 ), G )
    N_i = tf.multiply(S, tf.multiply(S2, tf.multiply(H, C)) )
    return N_i

In [9]:

#  linear system 

def matrix():
    global A, B
    A = np.repeat(alfa, st).reshape(st, st).T       # Matriz de parâmetros    
    B = np.repeat(alfa, st).reshape(st, st).T        # Matriz de alfas 
    for i in range(st):
        for j in range(st):
            if i==j:
                A[i, j] = 1 - (A[i, j] - 1)*(theta*lamb + gamma1)
                B[i, j] = B[i, j] - 1
            if i != j:
                A[i, j] = -A[i, j]*(theta*lamb + gamma1) 
    A = tf.convert_to_tensor(A, tf.float64)
    B = tf.convert_to_tensor(B, tf.float64)
    

def lin_sys(x1):
    #global N_i
    #print(f'x1 = {x1}')
    N_i = N_if(x1)
    N_i2 = tf.math.log( N_i )                                                    # Logaritmo de N_i
    C = tf.math.log(alfa)                                                        # Logaritmo dos alfas
    D = tf.reshape(tf.tensordot(B, tf.reshape(N_i2, (3,1)) , 1), (1, 3) ) + C    # produto matricial                     
    res = tf.reshape(tf.exp(tf.linalg.solve(A, tf.reshape(D, (3,1)) ) ), (1,3) )  # Resolvendo o sistema  A w = D
    #res =tf.reshape(res, (1,3))
    return res

In [10]:
def w_til_f(x1):
    S = tf.pow(s, phi)
    S2 = tf.pow( (1 - s), 1/(beta*kappa) )
    H = 1 + x1[0]
    w_til = tf.multiply(psi, tf.multiply(H, tf.multiply(S, tf.multiply(w , S2))))
    return w_til

   

In [11]:
# wages - model 

def W_i_f(x1):
    w_til = w_til_f(x1)
    sum_wtil = tf.pow( tf.reduce_sum( tf.pow(w_til, theta) ) , (kappa/theta) ) 
    S = tf.pow((1-s), (-1/beta))
    p_i = tf.pow(w_til, theta)/ tf.reduce_sum( tf.pow(w_til, theta) )   
    W_i = tf.multiply(kappa, tf.multiply(G, tf.multiply(S, tf.multiply(sum_wtil, p_i)) ) )    
    return W_i, p_i

In [12]:
# human capital

def hc_f(x1):
    W = tf.pow(w , (theta*lamb + gamma1) )
    W_til = tf.pow(tf.reduce_sum(tf.pow(w_til_f(x1), theta) ), lamb)
    N_i = N_if(x1)
    Hc = tf.multiply(N_i, tf.divide(W, W_til))
    return Hc

In [13]:
# PNAD - DATA


def get_p_i(year):
    global p_s2
    w='data_' + str(year)
    p_i = tf.convert_to_tensor(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,2]).reshape(1,3), tf.float64)
    p_s2 = p_i[1:st]
    return p_i


def get_W_i(year):
    p_i = get_p_i(year)
    global W_id
    w='data_' + str(year)
    W_id = tf.convert_to_tensor(np.multiply(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,1]), p_i).reshape(1,3), tf.float64)
    return W_id

In [14]:
@tf.function
def tflow(x1):    
    global w     
    w = lin_sys(x1)
    x1 = tf.reshape(x1, (1, 3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    #D = tf.math.log(D)
    return D

In [15]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [16]:
w

In [17]:
np.array(w)

In [18]:
tf.convert_to_tensor(w, tf.float64)

In [19]:
# bounds


Bd =   ((-10.0, 10.0), )*st 

#Bd = (tf.constant(-20.0, dtype=tf.float64), tf.constant(20.0, dtype=tf.float64), )*st

Bd = np.array(Bd, dtype=object)

In [20]:
# callback


cc = 0
def callback(x):
    global cc
    cc += 1
    fobj = tflow(x)
    print(f'\033[1;033mObjetivo: {np.around(fobj, 10)}, iter: {cc}') 

In [21]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0



dtype= tf.float64
start = tf.constant(np.array([1.0, 3.10, -1.20]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [22]:
tflow([1.2, 0.0, 1.0])

In [23]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [24]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [25]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [26]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [27]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [28]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [29]:
w

In [30]:
# taus
st=3
def taus(st):        
    x1 = tf.random.uniform(maxval=10, minval=-10, shape=[1, st], dtype='float64')
    return x1


def taus(st):
    tau_h = tf.random.uniform(maxval=10, minval=-10, shape=[1, st], dtype='float64')
    x = np.array([tau_h])
    x = tf.convert_to_tensor(x, tf.float64)
    return x
    

In [31]:
taus(3)

In [32]:
# taus



def taus(st):
    tau_h = tf.random.uniform(maxval=10, minval=-10, shape=[1, st], dtype='float64')
    x = np.array([tau_h])
    x = tf.convert_to_tensor(x, tf.float64)
    return x
    

In [33]:
taus(3)

In [34]:
taus(3)

In [35]:
taus(3)

In [36]:
taus(3)

In [37]:
taus(3)

In [38]:
taus(3)

In [39]:
taus(3)

In [40]:
taus(3)

In [41]:
taus(3)

In [42]:
taus(3)

In [43]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [44]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [45]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [46]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [47]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [48]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [49]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [50]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [51]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [52]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [53]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [54]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [55]:
w

In [56]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [57]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [58]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [59]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [60]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [61]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [62]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [63]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [64]:
w

In [65]:
@tf.function
def tflow(x1):    
    #global w     
    w = lin_sys(x1)
    x1 = tf.reshape(x1, (1, 3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    #D = tf.math.log(D)
    return D

In [66]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [67]:
@tf.function
def tflow(x1):    
    global w     
    w = lin_sys(x1)
    x1 = tf.reshape(x1, (1, 3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    #D = tf.math.log(D)
    return D

In [68]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [69]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [70]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [71]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [72]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [73]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [74]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [75]:
#@tf.function
def tflow(x1):    
    global w     
    w = lin_sys(x1)
    x1 = tf.reshape(x1, (1, 3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    #D = tf.math.log(D)
    return D

In [76]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [77]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [78]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [79]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [80]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [81]:
w

In [82]:
@tf.function
def tflow(x1):    
    global w     
    w = lin_sys(x1)
    x1 = tf.reshape(x1, (1, 3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    #D = tf.math.log(D)
    return D

In [83]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [84]:
w

In [85]:
#@tf.function
def tflow(x1):    
    global w     
    w = lin_sys(x1)
    x1 = tf.reshape(x1, (1, 3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    #D = tf.math.log(D)
    return D

In [86]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [87]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [88]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [89]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [90]:
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [91]:
w

In [92]:
sol = differential_evolution(tflow, Bd, strategy='best1exp',
                             tol=0,
                             polish=True, 
                             popsize=100, 
                             maxiter=100,
                             recombination =0.7,
                             disp=True)

In [93]:
print(f'Objective function: {sol.fun}' )
print(f'Optimization vectors: {sol.x.reshape(1,3)[0]}')



#-----------------  (-10, 10)

# 9.37196472 8.44134184 8.66419531 ----> 15
# 9.37196472 8.44134184 8.66419531 ---> 02

In [94]:
d = sol.x.reshape(1, st)

p_m = W_i_f(d)[1]
p_d = get_p_i(ano)

W_m = W_i_f(d)[0]
W_d = get_W_i(ano)


fig,ax = plt.subplots(1, 2, figsize=(14,8))
ax[0].scatter(p_m, p_d, s=100, color='red')
ax[0].plot([0.0, 0.7], [0.0, 0.7], 'k-', lw=2, label='linha de 45°', color='blue')
ax[0].grid(True)
ax[0].set_title(r'$p_i$', fontsize=20)
ax[0].tick_params(axis='both', which='major', labelsize=20)

ax[1].scatter(W_m, W_d, s=100, color='red')
ax[1].plot([np.min(W_m), np.max(W_m)], [np.min(W_m), np.max(W_m)], 'k-', lw=2, label='linha de 45°', color='blue')
ax[1].grid(True)
ax[1].set_title(r'$W_i$', fontsize=20)
ax[1].tick_params(axis='both', which='major', labelsize=20)

In [95]:
w

In [96]:
st = 3
x1 = taus(st)
ano= 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

In [97]:
sol = differential_evolution(tflow, Bd, strategy='best1exp',
                             tol=0,
                             polish=True, 
                             popsize=100, 
                             maxiter=100,
                             recombination =0.7,
                             disp=True)

In [98]:
w

In [99]:
d = sol.x.reshape(1, st)

p_m = W_i_f(d)[1]
p_d = get_p_i(ano)

W_m = W_i_f(d)[0]
W_d = get_W_i(ano)


fig,ax = plt.subplots(1, 2, figsize=(14,8))
ax[0].scatter(p_m, p_d, s=100, color='red')
ax[0].plot([0.0, 0.7], [0.0, 0.7], 'k-', lw=2, label='linha de 45°', color='blue')
ax[0].grid(True)
ax[0].set_title(r'$p_i$', fontsize=20)
ax[0].tick_params(axis='both', which='major', labelsize=20)

ax[1].scatter(W_m, W_d, s=100, color='red')
ax[1].plot([np.min(W_m), np.max(W_m)], [np.min(W_m), np.max(W_m)], 'k-', lw=2, label='linha de 45°', color='blue')
ax[1].grid(True)
ax[1].set_title(r'$W_i$', fontsize=20)
ax[1].tick_params(axis='both', which='major', labelsize=20)

In [100]:
A

In [101]:
d = sol.x.reshape(2, st)


h=hc_f(d)

In [102]:
d = sol.x.reshape(1, st)


h=hc_f(d)

In [103]:
cd = 15
np.round((alfa/h)*np.prod(np.power(h, alfa) ), cd) == np.round(lin_sys(d), cd

In [104]:
cd = 15
np.round((alfa/h)*np.prod(np.power(h, alfa) ), cd) == np.round(lin_sys(d), cd)

In [105]:
cd = 14
np.round((alfa/h)*np.prod(np.power(h, alfa) ), cd) == np.round(lin_sys(d), cd)

In [106]:
cd = 12
np.round((alfa/h)*np.prod(np.power(h, alfa) ), cd) == np.round(lin_sys(d), cd)

In [107]:
cd = 10
np.round((alfa/h)*np.prod(np.power(h, alfa) ), cd) == np.round(lin_sys(d), cd)

In [108]:
cd = 10
np.round((alfa/h)*np.prod(np.power(h, alfa) ), cd)
np.round(lin_sys(d), cd)

In [109]:
np.round((alfa/h)*np.prod(np.power(h, alfa) ), cd)

In [110]:
sol = differential_evolution(tflow, Bd, strategy='best1exp',
                             tol=0,
                             polish=True, 
                             popsize=100, 
                             maxiter=500,
                             recombination =0.7,
                             disp=True)